##      Setup and Installation

In [2]:
%pip install nltk pandas

import nltk
import pandas as pd
import random

# Download the VADER lexicon (only needed on the first run)
nltk.download('vader_lexicon')

# Import and create an instance of the VADER analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

print("✅ Setup Complete!")

Note: you may need to restart the kernel to use updated packages.
✅ Setup Complete!


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\yashu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


##  Load and Prepare the Song Database

In [3]:
# Load the main dataset
try:
    df = pd.read_csv('data.csv')
    
    # Clean up the 'artists' column to make it more readable
    # It's currently a string that looks like a list, e.g., "['Artist Name']"
    # We will remove the brackets and quotes.
    df['artists'] = df['artists'].str.replace(r"[\[\]']", "", regex=True)

    print("🎵 Song Database Loaded Successfully!")
    print(f"Total songs in the database: {len(df)}")
    
except FileNotFoundError:
    print("❌ Error: 'data.csv' not found. Please make sure the file is uploaded.")

🎵 Song Database Loaded Successfully!
Total songs in the database: 170653


##  Define the Database-Driven Recommendation Function

In [4]:
def get_song_suggestion_from_db(text):

    score_dict = sia.polarity_scores(text)
    compound_score = score_dict['compound']


    # Determine the sentiment category and corresponding valence range
    if compound_score <= -0.6:
        sentiment = "Very Sad 😭"
        valence_range = (0, 0.2)
    elif -0.6 < compound_score <= -0.2:
        sentiment = "Sad 😔"
        valence_range = (0.2, 0.4)
    elif -0.2 < compound_score < 0.2:
        sentiment = "Neutral 😐"
        valence_range = (0.4, 0.6)
    elif 0.2 <= compound_score < 0.6:
        sentiment = "Happy 😊"
        valence_range = (0.6, 0.8)
    else: # compound_score >= 0.6
        sentiment = "Very Happy 😄"
        valence_range = (0.8, 1.0)
        

    # Filter the DataFrame to find songs in the desired valence range
    filtered_songs = df[(df['valence'] >= valence_range[0]) & (df['valence'] <= valence_range[1])]
    

    # If songs are found, pick a random one. Otherwise, handle the case where no songs match.
    if not filtered_songs.empty:
        # Select a random song from the filtered list
        suggested_song = filtered_songs.sample(n=1).iloc[0]
        song_info = {
            "artist": suggested_song['artists'],
            "title": suggested_song['name']
        }
    else:
        song_info = {"artist": "N/A", "title": "No song found for this mood in the database."}


    return {
        "sentiment": sentiment,
        "score": compound_score,
        "song": song_info
    }

print("🧠 Database-driven analysis function is ready.")

🧠 Database-driven analysis function is ready.


## User Input and Final Output

In [7]:
# Get input from the user
user_text = input("How are you feeling today? Describe it in a few words: ")

# Get the analysis and suggestion from the database
result = get_song_suggestion_from_db(user_text)

# Print the results in a user-friendly format
print("\n-------------------------------------")
print(f"Sentiment Analysis: {result['sentiment']}")
print(f"Sentiment Score: {result['score']:.2f}")
print("-------------------------------------")
print(f"✨ Song Suggestion For You ✨")
print(f"'{result['song']['title']}' by {result['song']['artist']}")
print("-------------------------------------")


-------------------------------------
Sentiment Analysis: Neutral 😐
Sentiment Score: 0.13
-------------------------------------
✨ Song Suggestion For You ✨
'Gonna Fly Now' by Bill Conti
-------------------------------------
